In [ ]:
from datetime import datetime
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr

In addition we want to pull in a couple of local functions.
View the README to make sure you have it installed correctly.

In [ ]:
from codebase import BoundingBox, TimeRange
from codebase.namingconventions import get_nc_fname

We can define the starting and end times of our analysis

In [ ]:
trange = TimeRange(datetime(2017, 8, 25, 0), datetime(2017, 8, 28, 23))

We need to specify where we have the data stored.
In my case, it's in the following location:

In [ ]:
data_dir = "/Volumes/research/jd82/nexrad-xarray/data/external/"

Now we can read in all the files using the [`xr.open_mfdataset`](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html) (multi-file dataset) function.

In [ ]:
ds = (
    xr.open_mfdataset(
        [get_nc_fname(dt, dirname=data_dir) for dt in trange.dts],
        combine="nested",
        concat_dim="time",
        chunks={"time": 10},
    )
    .sortby("time")
    .rename({"param9.6.209": "prcp_rate"})["prcp_rate"]
)

This isn't working now because the dataset is still being loaded in!

Now we can convert to daily data for plotting

In [ ]:
daily = (
    ds.resample(time="1D")  # resample to daily (1D = 1 day) -- like a groupby
    .sum(skipna=False)  # take the daily sum
    .rename({"time": "date"})
)

And build the plot.
See [xarrray docs](https://docs.xarray.dev/en/stable/user-guide/plotting.html) for details.

In [ ]:
p = daily.plot(
    transform=ccrs.PlateCarree(),
    col="date",
    col_wrap=2,
    subplot_kws={"projection": ccrs.PlateCarree()},
    figsize=(12, 6),
)

for ax in p.axes.flat:
    ax.add_feature(cfeature.STATES)

plt.show()